In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mplt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.core.interactiveshell import InteractiveShell
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
import math
import gc
import ipaddress
from urllib.parse import urlparse
from tldextract import extract
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils.dataframe import column as column_utils


from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report

import lightgbm as lgb

np.set_printoptions(threshold=np.nan)



plt.rcParams["figure.figsize"] = (24,4)

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
import missingno as msno
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
import datetime
from scipy import signal
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn import linear_model



InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_seq_items', None)
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import warnings
warnings.filterwarnings('ignore')
import dask.dataframe as dd

In [2]:
df_train = pd.read_hdf("data/train_1.h5","table")
gc.collect()


11

In [3]:
df_test = pd.read_hdf("data/test_1.h5","table")

In [4]:
GROUP_BY_NEXT_CLICKS = [
    
    # V1
    {'groupby': ['ip']},
    {'groupby': ['app']},
    {'groupby': ['device']},
    {'groupby': ['ip', 'app']},
    {'groupby': ['ip', 'channel']},
    {'groupby': ['ip', 'os']},
    
    # V3
    {'groupby': ['ip', 'app', 'device', 'os', 'channel']},
    {'groupby': ['ip', 'os', 'device']},
    {'groupby': ['ip', 'os', 'device', 'app']},
    
    {'groupby': ['ip','hour']},
    {'groupby': ['ip', 'app', 'device', 'os', 'channel','hour']},
    {'groupby': ['ip', 'os', 'device','hour']},
    {'groupby': ['ip', 'os', 'device', 'app','hour']},
    {'groupby': ['ip', 'app','hour']},
    {'groupby': ['ip', 'channel','hour']},
    {'groupby': ['ip', 'os','hour']},
    
    {'groupby': ['app', 'device', 'os', 'channel']},
    {'groupby': ['app', 'device', 'os']},
    
    {'groupby': ['app', 'device', 'os','hour']},
]

In [5]:


def add_time_to_next_click(GROUP_BY_NEXT_CLICKS):
    gc.collect()
    # Calculate the time to next click for each group
    for spec in GROUP_BY_NEXT_CLICKS:

        # Name of new feature
        new_feature = '{}_nextClick'.format('_'.join(spec['groupby']))    

        # Unique list of features to select
        all_features = spec['groupby'] + ['click_time']

        # Run calculation
        print(f">> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}")
        group = df_train[all_features].groupby(spec['groupby']).click_time  
        gp = group.transform(lambda x: x.diff().shift(-1)).dt.seconds

        df_train[new_feature] = gp
        
        gp=None
        del gp
        
        group = df_test[all_features].groupby(spec['groupby']).click_time  
        gp = group.transform(lambda x: x.diff().shift(-1)).dt.seconds

        df_test[new_feature] = gp
        
        gp=None
        del gp
        group = None
        del group
        gc.collect()
    


In [ ]:
add_time_to_next_click(GROUP_BY_NEXT_CLICKS)

>> Grouping by ['ip'], and saving time to next click in: ip_nextClick
>> Grouping by ['app'], and saving time to next click in: app_nextClick
>> Grouping by ['device'], and saving time to next click in: device_nextClick
>> Grouping by ['ip', 'app'], and saving time to next click in: ip_app_nextClick
>> Grouping by ['ip', 'channel'], and saving time to next click in: ip_channel_nextClick
>> Grouping by ['ip', 'os'], and saving time to next click in: ip_os_nextClick
>> Grouping by ['ip', 'app', 'device', 'os', 'channel'], and saving time to next click in: ip_app_device_os_channel_nextClick
>> Grouping by ['ip', 'os', 'device'], and saving time to next click in: ip_os_device_nextClick
>> Grouping by ['ip', 'os', 'device', 'app'], and saving time to next click in: ip_os_device_app_nextClick
>> Grouping by ['ip', 'hour'], and saving time to next click in: ip_hour_nextClick
>> Grouping by ['ip', 'app', 'device', 'os', 'channel', 'hour'], and saving time to next click in: ip_app_device_os_cha

In [ ]:
gc.collect()
df_test.to_hdf('data/test_2.h5',"table",mode='w')
gc.collect()
print("Test_2.h5 done")

In [ ]:
df_test.head()
df_test = None
gc.collect()
del df_test
gc.collect()
df_train.head()

In [ ]:
gc.collect()
df_train.to_hdf('data/train_2.h5',"table",mode='w')
gc.collect()
print("Train_2.h5 done")

In [ ]:
df_utils.count_nulls(df_train)

In [122]:
gc.collect()

18